# Проект по SQL

**Описание проекта**
- Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Данные**
- Для анализа дана база данных,в ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Цель**
- Проанализировать базу данных и сформулировать ценностное предложение для нового продукта для нового приложения с электронными книгами.

**Задачи**
- 1. Посчитать, сколько книг вышло после 1 января 2000 года;
- 2. Для каждой книги посчитайть количество обзоров и среднюю оценку;
- 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- 4. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Ход исследования**
- Подключимся к базе данных
- Просмотрим таблицы
- С помощью SQL запросов проанализируем данные в соответсвиии с задачами
- Вывод

**Описание данных**

**Таблица books**
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews**
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

### Импорт библиотек и подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Просмотр содержания таблиц

- Таблица books

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 3'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT column_name, column_default, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'books' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,column_name,column_default,data_type
0,book_id,None,integer
1,author_id,None,integer
2,num_pages,None,integer
3,publication_date,None,date
4,publisher_id,None,integer
5,title,None,text


Все в порядке. Пропусков нет, форматы данных корректны, только вместо varchar - text (видимо, чтобы не было ограничений по длине в строке), это не критично. У колонки publication_date формат date, не datetime, как заявлено

- Таблица authors

In [5]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM authors LIMIT 3'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


In [6]:
# тип данных и пропуски
query = '''SELECT column_name, column_default, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'authors' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,column_name,column_default,data_type
0,author_id,None,integer
1,author,None,text


Все в порядке. Пропусков нет, форматы данных корректны, снова вместо varchar->text. Не критично.

- Таблица ratings

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM ratings LIMIT 3'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [8]:
# тип данных и пропуски
query = '''SELECT column_name, column_default, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'ratings' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,column_name,column_default,data_type
0,rating_id,None,integer
1,book_id,None,integer
2,rating,None,integer
3,username,None,text


Все в порядке

- Таблица reviews

In [9]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM reviews LIMIT 3'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


In [10]:
# тип данных и пропуски
query = '''SELECT column_name, column_default, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'reviews' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,column_name,column_default,data_type
0,review_id,None,integer
1,book_id,None,integer
2,username,None,text
3,text,None,text


Все в порядке

- Таблица publishers

In [11]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM publishers LIMIT 3'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


In [12]:
# тип данных и пропуски
query = '''SELECT column_name, column_default, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'publishers' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,column_name,column_default,data_type
0,publisher_id,None,integer
1,publisher,None,text


**Вывод по просмотру данных** С данными все в порядке, пропусков нет, столбцы соответсвуют заявленным. С типами данных тоже все в порядке. Столбцы с текстом имеют формат text вместо varchar (видимо, из-за большой длины строки)

### Анализ данных
- **Посчитаем, сколько книг вышло после 1 января 2000 года**

In [13]:
# чтобы выполнить SQL-запрос, используем Pandas
query_1 = '''SELECT COUNT(DISTINCT book_id) FROM books b WHERE CAST(publication_date AS date) > '2000-01-01' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_1), con = con)

,count
0,819


С января 2000 года было издано всего 819 книг

- **Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [14]:
# чтобы выполнить SQL-запрос, используем Pandas
query_2 = '''SELECT title, reviews, avg_rating FROM books as b
LEFT JOIN (SELECT book_id, COUNT(review_id) as reviews 
FROM reviews GROUP BY  book_id) as rev ON b.book_id=rev.book_id
LEFT JOIN (SELECT book_id,  ROUND(AVG(rating),2) as avg_rating 
FROM ratings GROUP BY book_id ) as rat ON b.book_id=rat.book_id 
LIMIT 10'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_2), con = con)

,title,reviews,avg_rating
0,The Body in the Library (Miss Marple #3),2,4.50
1,Galápagos,2,4.50
2,A Tree Grows in Brooklyn,5,4.25
3,Undaunted Courage: The Pioneering First Missio...,2,4.00
4,The Prophet,4,4.29
5,American Gods (American Gods #1),5,4.00
6,Influence: The Psychology of Persuasion,3,4.33
7,The Idiot,3,4.25
8,Rebecca,4,4.00
9,The Source,2,3.50


In [15]:
#минимальный рейтинг книг
query_2_1 = '''SELECT title, reviews, avg_rating FROM books as b
LEFT JOIN (SELECT book_id, COUNT(review_id) as reviews 
FROM reviews GROUP BY  book_id) as rev ON b.book_id=rev.book_id
LEFT JOIN (SELECT book_id,  ROUND(AVG(rating),2) as avg_rating 
FROM ratings GROUP BY book_id ) as rat ON b.book_id=rat.book_id
ORDER BY avg_rating
LIMIT 2'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_2_1), con = con)

,title,reviews,avg_rating
0,Harvesting the Heart,2,1.5
1,His Excellency: George Washington,2,2.0


In [16]:
#максимальное количество обзоров
query_2_2 = '''SELECT book_id, COUNT(review_id) FROM reviews GROUP BY book_id ORDER BY COUNT(review_id) DESC LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_2_2), con = con)

,book_id,count
0,948,7


In [17]:
#минимальное количество обзоров
query_2_3 = '''SELECT book_id, COUNT(review_id) FROM reviews GROUP BY book_id ORDER BY COUNT(review_id) LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_2_3), con = con)

,book_id,count
0,638,1


- Мы создали таблицу с названиями книг и средним рейтингом и общим количеством обзоров. Опираясь на эти данные, можно определить список ТОП самых популярных книг среди читателей, чтобы включить их в свою базу. 
- Минимальное количество обзоров на книгу - 1, максимальное 7, рейтинг от 1.5 до 5 баллов. То есть ни одна книга, даже с самым низким рейтингом не осталась без обзора читателя. Учитывая это при разработке нужно обязательно включить отзывы/разделы рекомендации/рейтинги.

- **Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры**

In [18]:
# чтобы выполнить SQL-запрос, используем Pandas
query_3 = '''SELECT p.publisher, COUNT(b.book_id) as total_books
FROM books as b
LEFT JOIN publishers as p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY total_books DESC
LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_3), con = con)

,publisher,total_books
0,Penguin Books,42


Самое продуктивное издательство Penguin Books, которое издало 42 книги. В расчет были взяты только книги 50+ страниц, чтобы исключить из расчета брошюры.

 
- **Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками**

In [19]:
# чтобы выполнить SQL-запрос, используем Pandas
query_4 = '''SELECT author, AVG(rating) as rating FROM books as b
RIGHT JOIN (SELECT book_id
            FROM ratings
            GROUP BY book_id 
            HAVING COUNT(rating_id) > 49) as lim ON b.book_id=lim.book_id
LEFT JOIN authors a ON b.author_id = a.author_id
LEFT JOIN ratings r ON r.book_id = b.book_id
GROUP BY author
ORDER BY rating DESC 
LIMIT 1 '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_4), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling самый популярный писатель с высокими оценками, а значит все книги Гарри Поттера обязательно должны быть включены.

- **Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [20]:
#чтобы выполнить SQL-запрос, используем Pandas
query_5 = '''WITH users as 
(SELECT rev.username , count(review_id) as review
 FROM reviews AS rev
 RIGHT JOIN (SELECT username
             FROM ratings
             GROUP BY username HAVING COUNT(rating_id) > 48) AS rat ON rev.username = rat.username
GROUP BY rev.username)
SELECT AVG(review) from users'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query_5), con = con)

,avg
0,24.0


Пользователи, которые оставили более 48 оценок, написали отзыв-обзор в среднем о 24 книгах. Оценку можно поставить книге, которую не читал, но обзор составить на то, что не читал - не возможно. Поэтому можно считать, что в среднем активные пользователи прочли 24 книги.

### Вывод

- Мы создали таблицу с названиями книг и средним рейтингом и общим количеством обзоров. Опираясь на эти данные, можно определить список ТОП самых популярных книг среди читателей, чтобы включить их в свою базу.
- С января 2000 года было издано всего 819 книг
- Минимальное количество обзоров на книгу - 1, максимальное 7, рейтинг от 1.5 до 5 баллов. То есть ни одна книга, даже с самым низким рейтингом не осталась без обзора читателя. Учитывая это при разработке нужно обязательно включить отзывы/разделы рекомендации/рейтинги.
- Самое продуктивное издательство Penguin Books, которое издало 42 книги. В расчет были взяты только книги 50+ страниц, чтобы исключить из расчета брошюры.
- J.K. Rowling самый популярный писатель с высокими оценками, а значит все книги Гарри Поттера обязательно должны быть включены.
- Самые активные пользователи, которые оставили более 48 оценок, написали отзыв-обзор в среднем о 24 книгах. Оценку можно поставить книге, которую не читал, но обзор составить на то, что не читал - не возможно. Поэтому можно считать, что в среднем активные пользователи прочли 24 книги.

**Ценностное предложение**
- Провести анализ и выявить то, что пользуется спросом у читателей в наше время, чтобы включить это в свое приложение (выявить популярные книг, авторов и издательств на основе рейтингов пользователей)